# Deep Learning - 1

## Chapter 3: Multiple inputs and outputs Neural Network

### Let's bundle our code

---------------

### Import

In [1]:
import numpy as np

### Model

#### Neural network

<img src="images/layer.png" alt="Drawing" width="300"/>

In [2]:
class Layer:
    """Representing a neural network layer"""
    
    def __init__(self, n_inputs, n_outputs):
        """Initlize weights and bias"""
        #Mean=0 std = 1 68% -> [-1, 1]
        self.weights = np.random.randn(n_inputs, n_outputs)
        self.biases = np.zeros((1, n_outputs))
    
    def forward(self, inputs):
        """
        It multiplies the inputs by the weights 
        and then sums them, and then sums bias.
        """
        #To calculate gradient, remembering input values
        self.inputs = inputs
        #Calculate outputs' values
        self.outputs = np.dot(inputs, self.weights) + self.biases
    
    def backward(self, dvalues):
        """Gradient with respect to parameters"""
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)

### Loss

#### MSE

In [3]:
class Loss_MSE:
    """MSE Loss function"""
    
    def forward(self, y_pred, y_true):
        """Forward pass"""     
        error = np.mean((y_pred - y_true) ** 2)
        return error
    
    def backward(self, y_pred, y_true):
        """Derivative of MSE with respect to pred"""
        
        #Number of samples
        samples = len(y_pred)
        
        #Number of output nodes
        outputs = len(y_pred[0])
        
        #Derivative of MSE
        self.dresults = 2 * (y_pred - y_true) / (outputs * samples)

### Optimizer

#### Gradient descent 

In [4]:
class Optimizer_GD:
    """Gradient descent optimizer"""
    
    def __init__(self, alpha=1.):
        """Initialize hyperparameters"""
        self.alpha = alpha

    def update_parameters(self, layer):
        """Update parameters"""
        weights_delta = layer.dweights * self.alpha
        biases_delta = layer.dbiases * self.alpha
        
        #Update parameters
        layer.weights -= weights_delta
        layer.biases -= biases_delta

### Scaler

#### Standard Scaler

In [5]:
class Scaler_Standard:
    """Standard scaler"""
    
    def fit(self, data):
        """Find mean and std values"""
        self.means = data.mean(axis=0)
        self.stds = data.std(axis=0)
        return self
    
    def transform(self, data):
        """Transforming data"""
        return (data - self.means) / self.stds
    
    def fit_transform(self, data):
        """Fit and transform data"""
        return self.fit(data).transform(data)

#### MinMax Scaler

In [6]:
class Scaler_MinMax:
    """MinMax scaler"""
    
    def __init__(self, feature_range=(0,1)):
        """Initialize the feature range"""
        self.low, self.high = feature_range
    
    def fit(self, data):
        """Find min and max values"""
        self.min = data.min(axis=0)
        self.max = data.max(axis=0)
        return self
    
    def transform(self, data):
        """Transforming data"""
        data_std = (data - self.min) / (self.max - self.min)
        return data_std * (self.high - self.low) + self.low
    
    def fit_transform(self, data):
        """Fit and transform data"""
        return self.fit(data).transform(data)

#### Robust Scaler

In [7]:
class Scaler_Robust:
    """Robust scaler"""
    
    def fit(self, data):
        """Find median and iqr values"""
        self.medians = np.median(data, axis=0)
        self.p75, self.p25 = np.percentile(data, [75 ,25], axis=0)
        self.iqr = self.p75 - self.p25
        return self
    
    def transform(self, data):
        """Transforming data"""
        return (data - self.medians) / self.iqr
    
    def fit_transform(self, data):
        """Fit and transform data"""
        return self.fit(data).transform(data)

---------------

### Set Hyperparameters

In [8]:
max_epoch = 300
alpha = 0.1
batch_size = 2

### Construct Data

In [9]:
train_dataset = np.array([[30, 2],  
                          [25, 1],
                          [27, 3],
                          [23, 4]])
train_label = np.array([[1],
                        [2], 
                        [1], 
                        [3]])

### Data Pre-Processing

In [10]:
scaler = Scaler_Standard()
train_dataset = scaler.fit_transform(train_dataset)

In [11]:
train_dataset

array([[ 1.45010473, -0.4472136 ],
       [-0.48336824, -1.34164079],
       [ 0.29002095,  0.4472136 ],
       [-1.25675744,  1.34164079]])

### Initialize the model

In [12]:
layer1 = Layer(2,1)

### Initlize optimizer and loss function

In [13]:
loss_function = Loss_MSE()
optimizer = Optimizer_GD(alpha=alpha)

### Training the model

In [14]:
for epoch in range(max_epoch):
    #Forward pass
    layer1.forward(train_dataset)
    loss = loss_function.forward(layer1.outputs, train_label)
    print(f'epoch: {epoch} \tloss: {loss}')
    
    #Backward pass
    loss_function.backward(layer1.outputs, train_label)
    layer1.backward(loss_function.dresults)
    
    #Update parameters
    optimizer.update_parameters(layer1)

epoch: 0 	loss: 5.613878430332317
epoch: 1 	loss: 3.601833052442445
epoch: 2 	loss: 2.3640726671108765
epoch: 3 	loss: 1.5904103212725103
epoch: 4 	loss: 1.0991024409375743
epoch: 5 	loss: 0.7821853677929014
epoch: 6 	loss: 0.5746123255166299
epoch: 7 	loss: 0.4366297515071182
epoch: 8 	loss: 0.34359223187767873
epoch: 9 	loss: 0.280002255735254
epoch: 10 	loss: 0.23597721428765578
epoch: 11 	loss: 0.20512775218079168
epoch: 12 	loss: 0.18326697244884554
epoch: 13 	loss: 0.16761492519098095
epoch: 14 	loss: 0.1563020652976793
epoch: 15 	loss: 0.1480554299157472
epoch: 16 	loss: 0.14199785691985217
epoch: 17 	loss: 0.1375179753037586
epoch: 18 	loss: 0.13418502259727824
epoch: 19 	loss: 0.13169238085680252
epoch: 20 	loss: 0.12981971805366632
epoch: 21 	loss: 0.12840731653625578
epoch: 22 	loss: 0.1273384718892291
epoch: 23 	loss: 0.12652729420807043
epoch: 24 	loss: 0.12591016483856912
epoch: 25 	loss: 0.1254396929556683
epoch: 26 	loss: 0.12508039967841517
epoch: 27 	loss: 0.124805608

In [15]:
layer1.weights

array([[-0.72909818],
       [ 0.05171523]])

In [16]:
layer1.biases

array([[1.75]])